<a href="https://colab.research.google.com/github/garciafido/cima-goes/blob/master/Mesoscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q netCDF4
!pip install -q  pyproj
!pip install -q aioftp
!pip install -q aiofiles
# Cartopy
!apt-get -q install libproj-dev proj-data proj-bin
!apt-get -q install libgeos-dev
!pip install -q  cython
!pip install -q  cartopy

In [0]:
!pip uninstall -y -q cima.goes
!pip install -q git+https://github.com/garciafido/cima-goes.git
!pip show cima.goes

In [0]:
import json
from dataclasses import asdict

import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from cima.goes.utils import timeit
from cima.goes.storage import GCS, NFS, FTP
from cima.goes import ProductBand, Product, Band
from cima.goes.projects import BatchProcess, DatesRange, HoursRange
from cima.goes.tiles import get_data, get_lats_lons, LatLonRegion
from cima.goes.img import get_image_stream, stream2pil, CLOUD_TOPS_PALETTE


def show(image_stream):
    image = stream2pil(image_stream)
    plt.axis('off')
    plt.imshow(image)
    plt.show()


def save(image_stream, filepath, storage):
    storage.upload_stream(image_stream, filepath)


def data_vis(data):
    albedo = (data * np.pi * 0.3) / 663.274497
    return np.power(albedo, 1.5)


def data_ir(data):
    return data - 273


def process_minute(gcs,
                   year, month, day, hour, minute,
                   band_blobs):
    title = f'{year}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}'
    print('Begin:', title)

    storage, base_path = use_ftp()

    dataset = gcs.get_dataset(band_blobs[(Product.RadM, 2)])
    try:
        data = get_data(dataset)
        data = data_vis(data)
        lats, lons = get_lats_lons(dataset)
    finally:
        dataset.close()

    image_stream = get_image_stream(
        data, lats, lons,
        vmin=0,
        vmax=0.7,
        cmap='gray')
    filename = f'{year:04d}_{month:02d}_{day:02d}_{hour:02d}_{minute:02d}_VIS.png'
    filepath = f'{base_path}/{year:04d}/{month:02d}/{day:02d}/{hour:02d}/{filename}'
    save(image_stream, filepath, storage)
    region = LatLonRegion(
        lat_north=np.max(lats),
        lat_south=np.min(lats),
        lon_west=np.min(lons),
        lon_east=np.max(lons)
    )
    jsonpath = filepath[:-8] + '_region.json'
    storage.upload_data(bytes(json.dumps(asdict(region), indent=2), 'utf8'), jsonpath)
    #     show(image_stream)

    dataset = gcs.get_dataset(band_blobs[(Product.CMIPM, 13)])
    try:
        data = get_data(dataset)
        data = data_ir(data)
        lats, lons = get_lats_lons(dataset)
    finally:
        dataset.close()
    image_stream = get_image_stream(
        data, lats, lons,
        vmin=-90,
        vmax=50,
        cmap=CLOUD_TOPS_PALETTE)
    filename = f'{year:04d}_{month:02d}_{day:02d}_{hour:02d}_{minute:02d}_IR.png'
    filepath = f'{base_path}/{year:04d}/{month:02d}/{day:02d}/{hour:02d}/{filename}'
    save(image_stream, filepath, storage)
#     show(image_stream)


def get_credentials(ftp):
    CREDENTIALS_FILEPATH = '/OTs/gcs_credentials.json'
    data = ftp.download_data(CREDENTIALS_FILEPATH)
    return json.loads(data.decode('UTF-8'))


def use_ftp():
    FTP_HOST = '<Your Server URL>'
    FTP_USER = '<Your User Name>'
    FTP_PASSWORD = '<Your Password>'

    return FTP(
        host=FTP_HOST,
        user=FTP_USER,
        password=FTP_PASSWORD), '/OTs/mesoscale/'


def use_nfs():
    return NFS(), '/content/mesoscale/'


@timeit
def run_batch():
    ftp, base_path = use_ftp()
    gcs = GCS(credentials_as_dict=get_credentials(ftp))

    range1 = DatesRange(
        date(2018, 11, 10),
        date(2018, 11, 10),
        [HoursRange(15, 18)],
        '15a18'
    )

    range2 = DatesRange(
        date(2018, 11, 10),
        date(2018, 11, 10),
        [HoursRange(19, 22)],
        '19a22'
    )

    MACHINE_NAME = 'COLAB_SAMPLE'
    batch = BatchProcess(
        gcs,
        [
            ProductBand(Product.CMIPM, 13, subproduct=2),
            ProductBand(Product.RadM, 2, subproduct=2),
        ],
        [range1, range2],
        ftp,
        '/OTs/logs/',
        MACHINE_NAME,
    )

    results = batch.run(process_minute, workers=2)
    print(results)


run_batch()
